In [9]:
import re
import pandas as pd
from metaphone import doublemetaphone

In [15]:
def read_kaggle_names():
    dfa = pd.read_csv("resources/kaggle_female_names.csv")[["name"]]
    dfm = pd.read_csv("resources/kaggle_male_names.csv")[["name"]]
    dfmn = pd.read_csv("resources/ine_male_names.csv")[["name"]]
    dffn = pd.read_csv("resources/ine_female_names.csv")[["name"]]
    
    dfms = pd.read_csv("resources/ine_surnames_freq_ge_100.csv")[["surname"]]
    dfms.rename(columns = {'surname': 'name'}, inplace = True)
    dfms = dfms[["name"]]
    
    vFullList = pd.concat([dfa, dfm, dfmn, dffn, dfms])
    #vFullList = pd.concat([dfa, dfm, dfmn, dffn])
    
    vFullList["name"] = vFullList["name"].str.upper()
    vFullList = vFullList.drop_duplicates()["name"].dropna().tolist()
    
    vListNames = [x for x in vFullList if " " not in x]
    # Se separan los nombres doobles como: "MARIA CARMEN" en "MARIA" y "CARMEN".
    # Se agregan los nombres que no existan en la lista principal.
    vLongNames = [x.split(" ") for x in vFullList if " " in x]
    for vArrays in vLongNames:
        vArrays = [x for x in vArrays if x not in vListNames]
        if len(vArrays) > 0:
            vListNames.extend(vArrays)
    # Evita que palabras como "DE", "LA", "SU" se agreguen al dataset.
    #vListNames = [x for x in vListNames if len(x) > 2]
    return pd.DataFrame(vListNames, columns=["NAME"])

vListNames = read_kaggle_names()
vListNames.head(3)

,NAME
0,MARIA
1,CARMEN
2,JOSEFA


In [16]:
cCONSONANTS = "BCDFGHJKLMNÑPQRSTVWXZ"
cREGEX_NOT_REPEATED = re.compile(r"(.)\1+")

def get_semantic_token(iToken):
    vTokenRegex = iToken.upper()
    vTokenRegex = vTokenRegex.translate(str.maketrans(cCONSONANTS, "C" * len(cCONSONANTS)))
    vTokenRegex = cREGEX_NOT_REPEATED.sub(r"\1", vTokenRegex)
    return vTokenRegex

vListNames["METAPHONE_A"] = vListNames["NAME"].apply(lambda x : doublemetaphone(x)[0])
vListNames["METAPHONE_B"] = vListNames["NAME"].apply(lambda x : doublemetaphone(x)[1] if len(doublemetaphone(x)[1]) > 0 else doublemetaphone(x)[0])
vListNames["REGEX_CV"] = vListNames["NAME"].apply(lambda x : get_semantic_token(x))
vListNames.head(3)

,NAME,METAPHONE_A,METAPHONE_B,REGEX_CV
0,MARIA,MR,MR,CACIA
1,CARMEN,KRMN,KRMN,CACEC
2,JOSEFA,JSF,HSF,COCECA


In [17]:
vListNames.to_csv("resources/export_full_dataset.csv", index = False)